## Import the modules needed

In [ ]:
import geopandas as gpd
import folium

## Load Json Files

In [ ]:
# Load the GeoJSON files
gdf_denmark = gpd.read_file('location\denmark.json')  # Denmark data
gdf_latvia = gpd.read_file('location\latvia.json')  # Latvia data

## User Selection

In [ ]:
# User selection variable
user_selection = 'Latvia'  # Change to 'Latvia' as needed

## Filter and plot the data

In [ ]:
# Filter the GeoDataFrames
nordsren_iii_vest = gdf_denmark[gdf_denmark['name'] == 'Nordsren III vest']
latvia_wind_farm = gdf_latvia  # Assuming you want to include all entries, adjust if needed

# Function to extract coordinates for plotting
def extract_coords(gdf):
    coords = []
    for geom in gdf.geometry:
        if geom.geom_type == 'MultiPolygon':
            for polygon in geom.geoms:
                exterior_coords = [[point[1], point[0]] for point in polygon.exterior.coords]
                coords.append(exterior_coords)
        elif geom.geom_type == 'Polygon':
            exterior_coords = [[point[1], point[0]] for point in geom.exterior.coords]
            coords.append(exterior_coords)
    return coords



# Create a map centered around Europe
europe_map = folium.Map(location=[54.5260, 15.2551], zoom_start=4)


# Based on user selection, extract the appropriate coordinates and add them to the map
if user_selection == 'Denmark':
    denmark_coords = extract_coords(nordsren_iii_vest)
    for polygon in denmark_coords:
        folium.Polygon(locations=polygon, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(europe_map)
elif user_selection == 'Latvia':
    latvia_coords = extract_coords(latvia_wind_farm)
    for polygon in latvia_coords:
        folium.Polygon(locations=polygon, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(europe_map)

# Display the map
europe_map